In [3]:
#Cragamos librerias
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
#Cargar desde un archivo .csv sin indice
#df = pd.read_csv('Albany.csv')
#df.head(5)


df = pd.read_csv('Datos_limpios_Irlanda.csv')
df =df.drop(labels=["Unnamed: 0"], axis=1)
df

last_scraped source                                               name  \
0         12/09/24     --                         Inishowen Artists' Retreat   
1         12/09/24     --  Cottage within the grounds Rathmullan House Hotel   
2         11/09/24     --                                      Melmore Crest   
3         11/09/24     --                                      Fairway Haven   
4         11/09/24     --                     Spacious Dublin City Apartment   
...            ...    ...                                                ...   
30638     12/09/24     --                                  ARAN SOUTH ROOM 2   
30639     12/09/24     --                      In the heartbeat of Connemara   
30640     12/09/24     --                                House near Salthill   
30641     12/09/24     --                                     Home in Galway   
30642     12/09/24     --                              One bedroom apartment   

                                             description     host_name  \
0      A beautiful 1800s Irish cottage with a working...          Andy   
1      Your best option to stay if you want to visit ...    Travelnest   
2      Welcome to your secluded luxury retreat by the...           Kai   
3      Fairway Haven is located close to Dunfanaghy G...          Jack   
4      Presenting our extremely spacious and comforta...         Denis   
...                                                  ...           ...   
30638  ARAN SOUTH ROOMS with their own private entran...         Maria   
30639  Enjoy the peace and quiet while taking in the ...  Linda Peadar   
30640  Your best option to stay if you want to visit ...      Michaela   
30641  Newly renovated cosy 3 bedroom house available...          Cora   
30642  Nice cost one bedroom apartment 10 minutes fro...        Sandra   

      host_since           host_location  \
0       21/11/18       Trenbane, Ireland   
1       05/01/22     Mexico City, Mexico   
2       10/12/15    Gremersdorf, Germany   
3       01/07/23        Lifford, Ireland   
4       29/03/17     Mexico City, Mexico   
...          ...                     ...   
30638   08/01/14  County Galway, Ireland   
30639   10/03/24     Mexico City, Mexico   
30640   01/07/13         Dublin, Ireland   
30641   22/04/23   Clarinbridge, Ireland   
30642   22/06/21     Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Ageing hippy, trained as aerospace engineer wh...      within an hour   
1      Hello, we are the Travelnest Team and we offer...      within an hour   
2      Hello! I am the owner of the house but live in...  within a few hours   
3      I am a social person, who likes to provide the...      within an hour   
4      I am a social person, who likes to provide the...        within a day   
...                                                  ...                 ...   
30638  I am a social person, who likes to provide the...  within a few hours   
30639  I am a social person, who likes to provide the...      within an hour   
30640   Things I can't live without....\r\n1) Love\r\...        within a day   
30641  I am a social person, who likes to provide the...      within an hour   
30642  I am a social person, who likes to provide the...      within an hour   

      host_response_rate  ... number_of_reviews_l30d review_scores_rating  \
0                   100%  ...                    2.0                 4.98   
1                    88%  ...                    0.0                 5.00   
2                   100%  ...                    1.0                 5.00   
3                   100%  ...                    1.0                 4.83   
4                    95%  ...                    0.0                 4.90   
...                  ...  ...                    ...                  ...   
30638               100%  ...                    7.0                 4.92   
30639               100%  ...                    6

**Convertimos a numericos**

In [5]:
#df['host_response_time'] = df['host_response_time'].map({'a few days or more':1, 'within an hour':2,'within a few hours':3, 'within a day':4})
df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df["host_response_rate"] = df["host_response_rate"].str.rstrip('%').astype(float) / 100
#df['room_type'] = df['room_type'].map({'Entire home/apt': 1,'Private room': 2,'Shared room': 3,'Hotel room': 4})
#df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100
df['host_verifications'] = df['host_verifications'].map({"['email', 'phone', 'work_email']":1, "['email', 'phone']":2, "['phone']": 3,"['phone', 'work_email']": 4, "['email']": 5,"[]": 6,"['email', 'work_email']": 7,"['work_email']": 8})
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': 1, 'f': 0})
df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0})
df['has_availability'] = df['has_availability'].map({'t': 1, 'f': 0})
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})


In [6]:
df['property_type'] = df['property_type'].replace({
    'Entire rental unit': 1,
    'Entire condo': 2,
    'Private room in rental unit': 3,
    'Entire home': 4,
    'Private room in condo': 5,
    'Entire townhouse': 6,
    'Private room in home': 7,
    'Entire villa': 8,
    'Entire loft': 9,
    'Private room in villa': 10,
    'Entire serviced apartment': 11,
    'Private room in townhouse': 12,
    'Room in hotel': 13,
    'Entire guesthouse': 14,
    'Private room in bed and breakfast': 15,
    'Room in aparthotel': 16,
    'Private room in guest suite': 17,
    'Private room in casa particular': 18,
    'Shared room in hotel': 19,
    'Private room in guesthouse': 20,
    'Tiny home': 21,
    'Shared room in hostel': 22,
    'Private room in serviced apartment': 23,
    'Private room in boat': 24,
    'Houseboat': 25,
    'Entire guest suite': 26,
    'Boat': 27,
    'Shared room in rental unit': 28,
    'Shared room in condo': 29,
    'Private room in loft': 30,
    'Private room': 31,
    'Private room in bungalow': 32,
    'Private room in hostel': 33,
    'Entire bungalow': 34,
    'Entire vacation home': 35,
    'Entire cabin': 36,
    'Private room in shipping container': 37,
    'Room in hostel': 38,
    'Camper/RV': 39,
    'Entire place': 40,
    'Barn': 41,
    'Casa particular': 42,
    'Entire cottage': 43,
    'Private room in hut': 44,
    'Shared room in bungalow': 45,
    'Private room in tiny home': 46,
    'Shared room in bed and breakfast': 47,
    'Private room in vacation home': 48,
    'Hut': 49,
    'Private room in barn': 50,
    'Minsu': 51,
    'Shipping container': 52,
    'Private room in cabin': 53,
    'Tower': 54
})


**Reemplazar valores**

_host_acceptance_rate_

In [7]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_acceptance_rate'])
unico

array(['0%', '1%', '10%', '100%', '11%', '12%', '13%', '14%', '15%',
       '16%', '17%', '18%', '19%', '2%', '20%', '21%', '22%', '23%',
       '24%', '25%', '26%', '27%', '28%', '29%', '3%', '30%', '31%',
       '32%', '33%', '34%', '35%', '36%', '37%', '38%', '39%', '4%',
       '40%', '41%', '42%', '43%', '44%', '45%', '46%', '47%', '48%',
       '49%', '5%', '50%', '51%', '52%', '53%', '54%', '55%', '56%',
       '57%', '58%', '59%', '6%', '60%', '61%', '62%', '63%', '64%',
       '65%', '66%', '67%', '68%', '69%', '7%', '70%', '71%', '72%',
       '73%', '74%', '75%', '76%', '77%', '78%', '79%', '8%', '80%',
       '81%', '82%', '83%', '84%', '85%', '86%', '87%', '88%', '89%',
       '9%', '90%', '91%', '92%', '93%', '94%', '95%', '96%', '97%',
       '98%', '99%'], dtype=object)

In [8]:
df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100

In [9]:
#Convierto una variable a dicotómica
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(
    lambda x: 1 if isinstance(x, (int, float)) and x > 0.80 else (0 if isinstance(x, (int, float)) and x <= 0.80 else 'Sin respuesta')
)

df

last_scraped source                                               name  \
0         12/09/24     --                         Inishowen Artists' Retreat   
1         12/09/24     --  Cottage within the grounds Rathmullan House Hotel   
2         11/09/24     --                                      Melmore Crest   
3         11/09/24     --                                      Fairway Haven   
4         11/09/24     --                     Spacious Dublin City Apartment   
...            ...    ...                                                ...   
30638     12/09/24     --                                  ARAN SOUTH ROOM 2   
30639     12/09/24     --                      In the heartbeat of Connemara   
30640     12/09/24     --                                House near Salthill   
30641     12/09/24     --                                     Home in Galway   
30642     12/09/24     --                              One bedroom apartment   

                                             description     host_name  \
0      A beautiful 1800s Irish cottage with a working...          Andy   
1      Your best option to stay if you want to visit ...    Travelnest   
2      Welcome to your secluded luxury retreat by the...           Kai   
3      Fairway Haven is located close to Dunfanaghy G...          Jack   
4      Presenting our extremely spacious and comforta...         Denis   
...                                                  ...           ...   
30638  ARAN SOUTH ROOMS with their own private entran...         Maria   
30639  Enjoy the peace and quiet while taking in the ...  Linda Peadar   
30640  Your best option to stay if you want to visit ...      Michaela   
30641  Newly renovated cosy 3 bedroom house available...          Cora   
30642  Nice cost one bedroom apartment 10 minutes fro...        Sandra   

      host_since           host_location  \
0       21/11/18       Trenbane, Ireland   
1       05/01/22     Mexico City, Mexico   
2       10/12/15    Gremersdorf, Germany   
3       01/07/23        Lifford, Ireland   
4       29/03/17     Mexico City, Mexico   
...          ...                     ...   
30638   08/01/14  County Galway, Ireland   
30639   10/03/24     Mexico City, Mexico   
30640   01/07/13         Dublin, Ireland   
30641   22/04/23   Clarinbridge, Ireland   
30642   22/06/21     Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Ageing hippy, trained as aerospace engineer wh...      within an hour   
1      Hello, we are the Travelnest Team and we offer...      within an hour   
2      Hello! I am the owner of the house but live in...  within a few hours   
3      I am a social person, who likes to provide the...      within an hour   
4      I am a social person, who likes to provide the...        within a day   
...                                                  ...                 ...   
30638  I am a social person, who likes to provide the...  within a few hours   
30639  I am a social person, who likes to provide the...      within an hour   
30640   Things I can't live without....\r\n1) Love\r\...        within a day   
30641  I am a social person, who likes to provide the...      within an hour   
30642  I am a social person, who likes to provide the...      within an hour   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    1.00  ...                     2.0                  4.98   
1                    0.88  ...                     0.0                  5.00   
2                    1.00  ...                     1.0                  5.00   
3                    1.00  ...                     1.0                  4.83   
4                    0.95  ...                     0.0                  4.90   
...                   ...  ...                     ...                   ...   
30638                1.00  ...                     7.0                  4.92   
30639                1.00 

_room_type_

In [10]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
unico

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [11]:
#Convierto una variable a dicotómica
df['room_type'] = df['room_type'].replace(["Shared room", "Private room"], 0)
df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)
df

/var/folders/cz/7wvs0p453l9_tw6cq3z5z51h0000gn/T/ipykernel_60942/508382164.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)


last_scraped source                                               name  \
0         12/09/24     --                         Inishowen Artists' Retreat   
1         12/09/24     --  Cottage within the grounds Rathmullan House Hotel   
2         11/09/24     --                                      Melmore Crest   
3         11/09/24     --                                      Fairway Haven   
4         11/09/24     --                     Spacious Dublin City Apartment   
...            ...    ...                                                ...   
30638     12/09/24     --                                  ARAN SOUTH ROOM 2   
30639     12/09/24     --                      In the heartbeat of Connemara   
30640     12/09/24     --                                House near Salthill   
30641     12/09/24     --                                     Home in Galway   
30642     12/09/24     --                              One bedroom apartment   

                                             description     host_name  \
0      A beautiful 1800s Irish cottage with a working...          Andy   
1      Your best option to stay if you want to visit ...    Travelnest   
2      Welcome to your secluded luxury retreat by the...           Kai   
3      Fairway Haven is located close to Dunfanaghy G...          Jack   
4      Presenting our extremely spacious and comforta...         Denis   
...                                                  ...           ...   
30638  ARAN SOUTH ROOMS with their own private entran...         Maria   
30639  Enjoy the peace and quiet while taking in the ...  Linda Peadar   
30640  Your best option to stay if you want to visit ...      Michaela   
30641  Newly renovated cosy 3 bedroom house available...          Cora   
30642  Nice cost one bedroom apartment 10 minutes fro...        Sandra   

      host_since           host_location  \
0       21/11/18       Trenbane, Ireland   
1       05/01/22     Mexico City, Mexico   
2       10/12/15    Gremersdorf, Germany   
3       01/07/23        Lifford, Ireland   
4       29/03/17     Mexico City, Mexico   
...          ...                     ...   
30638   08/01/14  County Galway, Ireland   
30639   10/03/24     Mexico City, Mexico   
30640   01/07/13         Dublin, Ireland   
30641   22/04/23   Clarinbridge, Ireland   
30642   22/06/21     Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Ageing hippy, trained as aerospace engineer wh...      within an hour   
1      Hello, we are the Travelnest Team and we offer...      within an hour   
2      Hello! I am the owner of the house but live in...  within a few hours   
3      I am a social person, who likes to provide the...      within an hour   
4      I am a social person, who likes to provide the...        within a day   
...                                                  ...                 ...   
30638  I am a social person, who likes to provide the...  within a few hours   
30639  I am a social person, who likes to provide the...      within an hour   
30640   Things I can't live without....\r\n1) Love\r\...        within a day   
30641  I am a social person, who likes to provide the...      within an hour   
30642  I am a social person, who likes to provide the...      within an hour   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    1.00  ...                     2.0                  4.98   
1                    0.88  ...                     0.0                  5.00   
2                    1.00  ...                     1.0                  5.00   
3                    1.00  ...                     1.0                  4.83   
4                    0.95  ...                     0.0                  4.90   
...                   ...  ...                     ...                   ...   
30638                1.00  ...                     7.0                  4.92   
30639                1.00 

_availability_365_

In [12]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['availability_365'])
unico

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [13]:
#Convierto una variable a dicotómica
df['availability_365'] = df['availability_365'].apply(
    lambda x: 1 if x == 365 else (0 if isinstance(x, (int, float)) and x < 365 else 'Sin respuesta')
)

df

last_scraped source                                               name  \
0         12/09/24     --                         Inishowen Artists' Retreat   
1         12/09/24     --  Cottage within the grounds Rathmullan House Hotel   
2         11/09/24     --                                      Melmore Crest   
3         11/09/24     --                                      Fairway Haven   
4         11/09/24     --                     Spacious Dublin City Apartment   
...            ...    ...                                                ...   
30638     12/09/24     --                                  ARAN SOUTH ROOM 2   
30639     12/09/24     --                      In the heartbeat of Connemara   
30640     12/09/24     --                                House near Salthill   
30641     12/09/24     --                                     Home in Galway   
30642     12/09/24     --                              One bedroom apartment   

                                             description     host_name  \
0      A beautiful 1800s Irish cottage with a working...          Andy   
1      Your best option to stay if you want to visit ...    Travelnest   
2      Welcome to your secluded luxury retreat by the...           Kai   
3      Fairway Haven is located close to Dunfanaghy G...          Jack   
4      Presenting our extremely spacious and comforta...         Denis   
...                                                  ...           ...   
30638  ARAN SOUTH ROOMS with their own private entran...         Maria   
30639  Enjoy the peace and quiet while taking in the ...  Linda Peadar   
30640  Your best option to stay if you want to visit ...      Michaela   
30641  Newly renovated cosy 3 bedroom house available...          Cora   
30642  Nice cost one bedroom apartment 10 minutes fro...        Sandra   

      host_since           host_location  \
0       21/11/18       Trenbane, Ireland   
1       05/01/22     Mexico City, Mexico   
2       10/12/15    Gremersdorf, Germany   
3       01/07/23        Lifford, Ireland   
4       29/03/17     Mexico City, Mexico   
...          ...                     ...   
30638   08/01/14  County Galway, Ireland   
30639   10/03/24     Mexico City, Mexico   
30640   01/07/13         Dublin, Ireland   
30641   22/04/23   Clarinbridge, Ireland   
30642   22/06/21     Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Ageing hippy, trained as aerospace engineer wh...      within an hour   
1      Hello, we are the Travelnest Team and we offer...      within an hour   
2      Hello! I am the owner of the house but live in...  within a few hours   
3      I am a social person, who likes to provide the...      within an hour   
4      I am a social person, who likes to provide the...        within a day   
...                                                  ...                 ...   
30638  I am a social person, who likes to provide the...  within a few hours   
30639  I am a social person, who likes to provide the...      within an hour   
30640   Things I can't live without....\r\n1) Love\r\...        within a day   
30641  I am a social person, who likes to provide the...      within an hour   
30642  I am a social person, who likes to provide the...      within an hour   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    1.00  ...                     2.0                  4.98   
1                    0.88  ...                     0.0                  5.00   
2                    1.00  ...                     1.0                  5.00   
3                    1.00  ...                     1.0                  4.83   
4                    0.95  ...                     0.0                  4.90   
...                   ...  ...                     ...                   ...   
30638                1.00  ...                     7.0                  4.92   
30639                1.00 

_host_response_time_

In [14]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
unico

array(['a few days or more', 'within a day', 'within a few hours',
       'within an hour'], dtype=object)

In [15]:
df['host_response_time']=df['host_response_time'].replace(["within a few hours", "within a day", "a few days or more", "unknown"], 0)
df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)
df

/var/folders/cz/7wvs0p453l9_tw6cq3z5z51h0000gn/T/ipykernel_60942/2343497725.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)


last_scraped source                                               name  \
0         12/09/24     --                         Inishowen Artists' Retreat   
1         12/09/24     --  Cottage within the grounds Rathmullan House Hotel   
2         11/09/24     --                                      Melmore Crest   
3         11/09/24     --                                      Fairway Haven   
4         11/09/24     --                     Spacious Dublin City Apartment   
...            ...    ...                                                ...   
30638     12/09/24     --                                  ARAN SOUTH ROOM 2   
30639     12/09/24     --                      In the heartbeat of Connemara   
30640     12/09/24     --                                House near Salthill   
30641     12/09/24     --                                     Home in Galway   
30642     12/09/24     --                              One bedroom apartment   

                                             description     host_name  \
0      A beautiful 1800s Irish cottage with a working...          Andy   
1      Your best option to stay if you want to visit ...    Travelnest   
2      Welcome to your secluded luxury retreat by the...           Kai   
3      Fairway Haven is located close to Dunfanaghy G...          Jack   
4      Presenting our extremely spacious and comforta...         Denis   
...                                                  ...           ...   
30638  ARAN SOUTH ROOMS with their own private entran...         Maria   
30639  Enjoy the peace and quiet while taking in the ...  Linda Peadar   
30640  Your best option to stay if you want to visit ...      Michaela   
30641  Newly renovated cosy 3 bedroom house available...          Cora   
30642  Nice cost one bedroom apartment 10 minutes fro...        Sandra   

      host_since           host_location  \
0       21/11/18       Trenbane, Ireland   
1       05/01/22     Mexico City, Mexico   
2       10/12/15    Gremersdorf, Germany   
3       01/07/23        Lifford, Ireland   
4       29/03/17     Mexico City, Mexico   
...          ...                     ...   
30638   08/01/14  County Galway, Ireland   
30639   10/03/24     Mexico City, Mexico   
30640   01/07/13         Dublin, Ireland   
30641   22/04/23   Clarinbridge, Ireland   
30642   22/06/21     Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Ageing hippy, trained as aerospace engineer wh...                   1   
1      Hello, we are the Travelnest Team and we offer...                   1   
2      Hello! I am the owner of the house but live in...                   0   
3      I am a social person, who likes to provide the...                   1   
4      I am a social person, who likes to provide the...                   0   
...                                                  ...                 ...   
30638  I am a social person, who likes to provide the...                   0   
30639  I am a social person, who likes to provide the...                   1   
30640   Things I can't live without....\r\n1) Love\r\...                   0   
30641  I am a social person, who likes to provide the...                   1   
30642  I am a social person, who likes to provide the...                   1   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    1.00  ...                     2.0                  4.98   
1                    0.88  ...                     0.0                  5.00   
2                    1.00  ...                     1.0                  5.00   
3                    1.00  ...                     1.0                  4.83   
4                    0.95  ...                     0.0                  4.90   
...                   ...  ...                     ...                   ...   
30638                1.00  ...                     7.0                  4.92   
30639                1.00 

_accommodates_

In [16]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['accommodates'])
unico

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [17]:
df['accommodates'] = df['accommodates'].replace([1, 2, 2.9], 0).replace([3, 4, 5, 6, 7], 1)
df

last_scraped source                                               name  \
0         12/09/24     --                         Inishowen Artists' Retreat   
1         12/09/24     --  Cottage within the grounds Rathmullan House Hotel   
2         11/09/24     --                                      Melmore Crest   
3         11/09/24     --                                      Fairway Haven   
4         11/09/24     --                     Spacious Dublin City Apartment   
...            ...    ...                                                ...   
30638     12/09/24     --                                  ARAN SOUTH ROOM 2   
30639     12/09/24     --                      In the heartbeat of Connemara   
30640     12/09/24     --                                House near Salthill   
30641     12/09/24     --                                     Home in Galway   
30642     12/09/24     --                              One bedroom apartment   

                                             description     host_name  \
0      A beautiful 1800s Irish cottage with a working...          Andy   
1      Your best option to stay if you want to visit ...    Travelnest   
2      Welcome to your secluded luxury retreat by the...           Kai   
3      Fairway Haven is located close to Dunfanaghy G...          Jack   
4      Presenting our extremely spacious and comforta...         Denis   
...                                                  ...           ...   
30638  ARAN SOUTH ROOMS with their own private entran...         Maria   
30639  Enjoy the peace and quiet while taking in the ...  Linda Peadar   
30640  Your best option to stay if you want to visit ...      Michaela   
30641  Newly renovated cosy 3 bedroom house available...          Cora   
30642  Nice cost one bedroom apartment 10 minutes fro...        Sandra   

      host_since           host_location  \
0       21/11/18       Trenbane, Ireland   
1       05/01/22     Mexico City, Mexico   
2       10/12/15    Gremersdorf, Germany   
3       01/07/23        Lifford, Ireland   
4       29/03/17     Mexico City, Mexico   
...          ...                     ...   
30638   08/01/14  County Galway, Ireland   
30639   10/03/24     Mexico City, Mexico   
30640   01/07/13         Dublin, Ireland   
30641   22/04/23   Clarinbridge, Ireland   
30642   22/06/21     Mexico City, Mexico   

                                              host_about  host_response_time  \
0      Ageing hippy, trained as aerospace engineer wh...                   1   
1      Hello, we are the Travelnest Team and we offer...                   1   
2      Hello! I am the owner of the house but live in...                   0   
3      I am a social person, who likes to provide the...                   1   
4      I am a social person, who likes to provide the...                   0   
...                                                  ...                 ...   
30638  I am a social person, who likes to provide the...                   0   
30639  I am a social person, who likes to provide the...                   1   
30640   Things I can't live without....\r\n1) Love\r\...                   0   
30641  I am a social person, who likes to provide the...                   1   
30642  I am a social person, who likes to provide the...                   1   

       host_response_rate  ...  number_of_reviews_l30d  review_scores_rating  \
0                    1.00  ...                     2.0                  4.98   
1                    0.88  ...                     0.0                  5.00   
2                    1.00  ...                     1.0                  5.00   
3                    1.00  ...                     1.0                  4.83   
4                    0.95  ...                     0.0                  4.90   
...                   ...  ...                     ...                   ...   
30638                1.00  ...                     7.0                  4.92   
30639                1.00 

In [18]:
df['availability_365']

0        0
1        0
2        0
3        0
4        0
        ..
30638    0
30639    0
30640    0
30641    0
30642    0
Name: availability_365, Length: 30643, dtype: int64

**REGRESIÓN LOGÍSTICA**

_host_is_superhost_

In [19]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_acceptance_rate', 'host_response_rate', 'number_of_reviews_ltm']]
Var_Dep = df['host_is_superhost']

In [20]:
X= Vars_Indep
y= Var_Dep

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=None)

In [22]:
escalar =StandardScaler()

In [23]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [24]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [25]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [26]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [27]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[10170  1410]
 [ 3728  3078]]


In [28]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6858288770053476


In [29]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7205482432285435


In [30]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.8782383419689119


_host_identity_verified_

In [31]:
#Declaramos las variables dependientes e independientes para regresión logistica
Vars_Indep=df[['has_availability', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['host_identity_verified']

In [32]:
df['host_acceptance_rate'].dtype

dtype('int64')

In [33]:
X1= Vars_Indep
y1= Var_Dep

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=None)

In [35]:
escalar =StandardScaler()

In [36]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [37]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [38]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [39]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 0, 0, ..., 1, 1, 1])

In [40]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 259  173]
 [  61 8700]]


In [41]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9805026484841655


In [42]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9745458501033395


In [43]:
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5995370370370371


_host_acceptance_rate_

In [44]:
Vars_Indep= df[["accommodates", "room_type","instant_bookable"]]
Var_Dep= df["host_acceptance_rate"]

In [45]:
X= Vars_Indep
y= Var_Dep

In [46]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None)

In [47]:
escalar= StandardScaler()

In [48]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [49]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [50]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [51]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [52]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[   0 1725]
 [   0 7468]]


In [53]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.8123572283259002


In [54]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.8123572283259002


In [55]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.0


_has_availability_

In [56]:
Vars_Indep=df[['host_identity_verified', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['has_availability']

In [57]:
df['host_identity_verified'].dtype

dtype('int64')

In [58]:
#Redefinimos las variables
X3 = Vars_Indep
y3 = Var_Dep

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.2, random_state=None)

In [60]:
escalar =StandardScaler()

In [61]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [62]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [63]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [64]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 0, 1, ..., 1, 1, 1])

In [65]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 191   44]
 [ 100 5794]]


In [66]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9924631723192874


In [67]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9765051395007343


In [68]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label= 0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.8127659574468085


_instant_bookable_

In [69]:
Vars_Indep= df[["accommodates", "room_type"]]
Var_Dep= df["instant_bookable"]

In [70]:
X= Vars_Indep
y= Var_Dep

In [71]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [72]:
escalar= StandardScaler()

In [73]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [74]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [75]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [76]:
y_pred= algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [77]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[6501    0]
 [2692    0]]


In [78]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [79]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.7071684977700424


In [80]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
1.0


_room_type_

In [81]:
Vars_Indep= df[["accommodates", "room_type", "instant_bookable"]]
Var_Dep= df["room_type"]

In [82]:
X= Vars_Indep
y= Var_Dep

In [83]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [84]:
escalar= StandardScaler()

In [85]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [86]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [87]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [88]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 0, 1, 1])

In [89]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[3053    0]
 [   0 6140]]


In [90]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
1.0


In [91]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
1.0


In [92]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
1.0


_accommodates_

In [93]:
Vars_Indep= df[[ "room_type", "instant_bookable"]]
Var_Dep= df["accommodates"]

In [94]:
X= Vars_Indep
y= Var_Dep

In [95]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [96]:
escalar= StandardScaler()

In [97]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [98]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [99]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [100]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1., 1., 0., ..., 1., 0., 0.])

In [101]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[2484 1239    0    0    0    0    0]
 [ 510 4063    0    0    0    0    0]
 [  14  481    0    0    0    0    0]
 [   6   98    0    0    0    0    0]
 [  10  180    0    0    0    0    0]
 [   6   45    0    0    0    0    0]
 [   4   53    0    0    0    0    0]]


In [103]:
from sklearn.metrics import precision_score

#precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
#print("predicciones del modelo")
#print(precision)


precision= precision_score(y_test, y_pred, average="weighted")
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.6597235339365689


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [104]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.7121723050146851


In [106]:
from sklearn.metrics import recall_score

#sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
#print("Sensibilidad del modelo: ")
#print(sensibilidad)

sensibilidad= recall_score(y_test, y_pred, average="weighted")
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.7121723050146851


_availability_365_

In [119]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_response_rate','price', 'review_scores_location', 'host_identity_verified']]
Var_Dep = df['availability_365']

In [120]:
df['review_scores_cleanliness'].dtype

dtype('float64')

In [121]:
df['host_response_rate'].dtype

dtype('float64')

In [122]:
df['price'].dtype

dtype('float64')

In [115]:
df['review_scores_location'].dtype

dtype('float64')

In [118]:
df['property_type'].dtype

dtype('O')

In [123]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [124]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [126]:
#Se escalan todos los datos
escalar = StandardScaler()

In [127]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)




In [128]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [129]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [130]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [131]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[8956    0]
 [ 237    0]]


In [132]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [134]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9742195148482541


In [135]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


_host_response_time_

In [136]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'host_total_listings_count', 'host_response_rate']]
Var_Dep = df['host_response_time']

In [137]:
df['host_response_time'].dtype
df['host_response_time'].drop_duplicates().tolist()

[1, 0]

In [138]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [139]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [140]:
#Se escalan todos los datos
escalar = StandardScaler()

In [141]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [142]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [143]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [144]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([0, 1, 1, ..., 1, 1, 1])

In [145]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[2053 1694]
 [ 397 5049]]


In [146]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.7487765089722676


In [147]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.7725443272054824


In [148]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5479049906591941


_host_has_profile_pic_

In [149]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'review_scores_rating', 'host_total_listings_count', 'host_is_superhost']]
Var_Dep = df['host_has_profile_pic']

In [150]:
df['review_scores_rating'].dtype

dtype('float64')

In [151]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [152]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [153]:
#Se escalan todos los datos
escalar = StandardScaler()

In [154]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [155]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [156]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [157]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [158]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   4  564]
 [  12 8613]]


In [159]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9385420071918927


In [160]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.937343631023605


In [161]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.007042253521126761
